In [1]:
import time
import numpy as np
import mxnet as mx
import os

os.environ['DGLBACKEND'] = 'mxnet'

import math
from mxnet import gluon, nd
from utils import collate, rmse_trainset, rmse_testset, GraphTraffic, \
    load_adjmatrix, EarlyStopping, train_seed, StandardScaler, masked_rmse_np, masked_mae_np, masked_mape_np
from model import SARGCN
import argparse
import warnings
import pickle

warnings.filterwarnings('ignore')

Using backend: mxnet


In [2]:
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def step(self, acc, model):
        score = acc
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score > self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0
        return self.early_stop

    def save_checkpoint(self, model):
        '''Saves model when validation loss decrease.'''
        model.save_parameters('SARGCN.param')

In [3]:
# random seed
train_seed(seed=12345)
device = mx.gpu(0)

In [4]:
def load_data():
    # load data
    g, tratim_matrix = load_adjmatrix('../data/adjacency_matrix_13.mat')
    edge_type = np.load('../data/edge_type_13.npy')
    edge_type = mx.nd.array(edge_type, ctx=device)

    # load data
    with open('../data/train.pkl', 'rb') as f:
        train = pickle.load(f)

    with open('../data/val.pkl', 'rb') as f:
        val = pickle.load(f)

    with open('../data/test.pkl', 'rb') as f:
        test = pickle.load(f)

    train_x, train_y = train['x'], train['y']
    valid_x, valid_y = val['x'], val['y']
    test_x, test_y = test['x'], test['y']

    train_x = np.swapaxes(train_x, 1, 2)
    train_x = np.concatenate([train_x[:, :, :, 0], train_x[:, :, :, 1]], axis=2)
    train_y = np.swapaxes(train_y, 1, 2)
    train_y = np.concatenate([train_y[:, :, :, 0], train_y[:, :, :, 1]], axis=2)

    valid_x = np.swapaxes(valid_x, 1, 2)
    valid_x = np.concatenate([valid_x[:, :, :, 0], valid_x[:, :, :, 1]], axis=2)
    valid_y = np.swapaxes(valid_y, 1, 2)
    valid_y = np.concatenate([valid_y[:, :, :, 0], valid_y[:, :, :, 1]], axis=2)

    test_x = np.swapaxes(test_x, 1, 2)
    test_x = np.concatenate([test_x[:, :, :, 0], test_x[:, :, :, 1]], axis=2)
    test_y = np.swapaxes(test_y, 1, 2)
    test_y = np.concatenate([test_y[:, :, :, 0], test_y[:, :, :, 1]], axis=2)

    scaler_axis = (0, 1, 2)
    scaler = StandardScaler(mean=train_x.mean(axis=scaler_axis),
                            std=train_x.std(axis=scaler_axis))
    train_x = nd.array(train_x, ctx=device)
    train_y = nd.array(train_y, ctx=device)
    valid_x = nd.array(valid_x, ctx=device)
    valid_y = nd.array(valid_y, ctx=device)
    test_x = nd.array(test_x, ctx=device)
    test_y = nd.array(test_y, ctx=device)

    train_x = scaler.transform(train_x)
    train_y = scaler.transform(train_y)
    valid_x = scaler.transform(valid_x)
    valid_y = scaler.transform(valid_y)
    test_x = scaler.transform(test_x)
    test_y = scaler.transform(test_y)

    num_train = train_x.shape[0]
    num_valid = valid_x.shape[0]
    num_test = test_x.shape[0]

    seq_len = 4
    pre_len = 4
    num_node = 80

    # divide training set, valid set and test set
    trainset = GraphTraffic(num_train, num_node, tratim_matrix)
    validset = GraphTraffic(num_valid, num_node, tratim_matrix)
    testset = GraphTraffic(num_test, num_node, tratim_matrix)

    for i in range(trainset.__len__()):
        trainset.graphs[i].ndata['h'] = train_x[i]
        trainset.graphs[i].edata['type'] = edge_type.reshape(-1, )
        trainset.labels[i] = train_y[i]
    for i in range(validset.__len__()):
        validset.graphs[i].ndata['h'] = valid_x[i]
        validset.graphs[i].edata['type'] = edge_type.reshape(-1, )
        validset.labels[i] = valid_y[i]
    for i in range(testset.__len__()):
        testset.graphs[i].ndata['h'] = test_x[i]
        testset.graphs[i].edata['type'] = edge_type.reshape(-1, )
        testset.labels[i] = test_y[i]

    return scaler, trainset, validset, testset

In [5]:
def main():
    scaler, trainset, validset, testset = load_data()
    dropout = 0.05
    in_feats = 2*4
    num_classes = 2*4
    lr = 1e-3
    weight_decay = 0.0
    n_bases = 1
    n_layers = 1
    n_epochs = 1
    num_rels = 25
    use_self_loop = True
    batch_size = 32
    test_batch_size = 32
    test_batch_size = 32
    n_hidden = 224
    n_channel = 96
    seq_len = 4
    num_nodes = 80

    # create model
    model = SARGCN(in_feats,
                   n_hidden,
                   num_classes,
                   num_rels,
                   inter_channel=n_channel,
                   num_bases=n_bases,
                   num_hidden_layers=n_layers,
                   dropout=dropout,
                   use_self_loop=use_self_loop,
                   gpu_id=mx.gpu(),
                   residual=False)
    model.initialize(mx.init.Xavier(magnitude=math.sqrt(2.0)), ctx=mx.gpu())
    trainer = gluon.Trainer(model.collect_params(),
                            'adam',
                            {'learning_rate': lr,
                             'wd': weight_decay})
    loss = gluon.loss.L2Loss()
    stopper = EarlyStopping(patience=15)
    train_iter = gluon.data.DataLoader(trainset, batch_size, shuffle=False,
                                       batchify_fn=collate, last_batch='discard',
                                       thread_pool=True)
    valid_iter = gluon.data.DataLoader(validset, test_batch_size, shuffle=False,
                                       batchify_fn=collate, last_batch='discard',
                                       thread_pool=True)
    test_iter = gluon.data.DataLoader(testset, test_batch_size, shuffle=False,
                                      batchify_fn=collate, last_batch='discard',
                                      thread_pool=True)
    TrainLoss, ValidLoss, TestLoss = [], [], []

    for epoch in range(1):
        train_l_sum, n = 0.0, 0
        start_time = time.time()
        for iter, (bg, label) in enumerate(train_iter):
            model.g = bg
            with mx.autograd.record():
                edge_type = bg.edata['type']
                pred = model(bg, bg.ndata['h'], edge_type, None)
                l = loss(pred, label)
            break
        break

    # evaluate
    model_list = ['pre_trained_model.param']

    for filename in model_list:
        print(filename)
        model.load_parameters(filename)
        test_rmse, test_mse, test_mae, test_mape, true, pred = rmse_testset(model, test_iter, batch_size, num_nodes, scaler, mae=True)
        for i in range(seq_len):
            rmse = masked_rmse_np(labels=true[:, :, [i, i+4]], preds=pred[:, :, [i, i+4]])
            mae = masked_mae_np(labels=true[:, :, [i, i+4]], preds=pred[:, :, [i, i+4]])
            mape = masked_mape_np(labels=true[:, :, [i, i+4]], preds=pred[:, :, [i, i+4]])
            print('time: %i, rmse: %.3f, mae: %.3f, mape: %.5f' % ((i+1)*15, rmse, mae, mape))

In [6]:
main()

number of nodes: 80
number of edges: 715
pre_trained_model.param
time: 15, rmse: 36.220, mae: 22.477, mape: 0.13942
time: 30, rmse: 37.825, mae: 23.464, mape: 0.14986
time: 45, rmse: 39.408, mae: 24.431, mape: 0.16254
time: 60, rmse: 41.589, mae: 25.294, mape: 0.17597
